In [1]:
import bayes3d as b
import bayes3d.genjax
import joblib
from tqdm import tqdm
import os
import jax.numpy as jnp
import jax
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
b.setup_visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7014/static/


In [3]:
intrinsics = b.Intrinsics(
    height=100,
    width=100,
    fx=500.0, fy=500.0,
    cx=50.0, cy=50.0,
    near=0.01, far=20.0
)

b.setup_renderer(intrinsics)
model_dir = os.path.join(b.utils.get_assets_dir(),"bop/ycbv/models")
meshes = []
for idx in range(1,22):
    mesh_path = os.path.join(model_dir,"obj_" + "{}".format(idx).rjust(6, '0') + ".ply")
    b.RENDERER.add_mesh_from_file(mesh_path, scaling_factor=1.0/1000.0)

b.RENDERER.add_mesh_from_file(os.path.join(b.utils.get_assets_dir(), "sample_objs/cube.obj"), scaling_factor=1.0/1000000000.0)


Increasing frame buffer size to (width, height, depth) = (128, 128, 1024)


[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


In [4]:
importance_jit = jax.jit(b.genjax.model.importance)
key = jax.random.PRNGKey(10)

In [5]:
gt_traces = []
NUM_TRACES = 20
for scene_id in tqdm(range(NUM_TRACES)):
    filename = f"data/trace_{scene_id}.joblib"
    gt_traces.append(importance_jit(key, *joblib.load(filename))[1][1])

hb_traces = []

V_VARIANT = 0
O_VARIANT = 0
HIERARCHICAL_BAYES = True

for scene_id in tqdm(range(NUM_TRACES)):
    if HIERARCHICAL_BAYES:
        filename = f"data/inferred_hb_{scene_id}.joblib"
    else:
        filename = f"data/inferred_{V_VARIANT}_{O_VARIANT}_{scene_id}.joblib"
    hb_traces.append(importance_jit(key, *joblib.load(filename))[1][1])

variant_0_0_traces = []

V_VARIANT = 0
O_VARIANT = 0
HIERARCHICAL_BAYES = False

for scene_id in tqdm(range(NUM_TRACES)):
    if HIERARCHICAL_BAYES:
        filename = f"data/inferred_hb_{scene_id}.joblib"
    else:
        filename = f"data/inferred_{V_VARIANT}_{O_VARIANT}_{scene_id}.joblib"
    variant_0_0_traces.append(importance_jit(key, *joblib.load(filename))[1][1])


100%|████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 23.80it/s]


In [9]:
# Classification accuracy
prediction_sets = [hb_traces]
wrong_prediction = []
for pred_set in prediction_sets:
    correct = 0
    for scene_id in range(NUM_TRACES):
        gt_ids = np.array(b.genjax.get_indices(gt_traces[scene_id]))
        pred_ids = np.array(b.genjax.get_indices(pred_set[scene_id]))
        # print(pred_set[scene_id]["variance"])
        # print(pred_set[scene_id]["outlier_prob"])
        if set(pred_ids) == set(gt_ids):
            correct +=1
        else:
            wrong_prediction.append(scene_id)
            print(gt_ids, pred_ids)
            continue
    print(correct)    

[21 15 18  6] [21 15  6 17]
[21 15 13  2] [21 15 13  4]
[21  5  9 21] [21  5  9 17]
[21 21  7 20] [21  7 20 17]
[21 13  1 18] [21 11 13 18]
[21 15 19 12] [21 12 19  2]
[21  1  3  1] [21  1  1 17]
13


In [10]:
wrong_prediction

[0, 3, 5, 11, 13, 15, 18]

In [11]:

importance_jit = jax.jit(b.genjax.model.importance)

contact_enumerators = [b.genjax.make_enumerator([f"contact_params_{i}", "variance", "outlier_prob"]) for i in range(5)]
add_object_jit = jax.jit(b.genjax.add_object)

def c2f_contact_update(trace_, key,  number, contact_param_deltas, VARIANCE_GRID, OUTLIER_GRID):
    contact_param_grid = contact_param_deltas + trace_[f"contact_params_{number}"]
    scores = contact_enumerators[number][3](trace_, key, contact_param_grid, VARIANCE_GRID, OUTLIER_GRID)
    i,j,k = jnp.unravel_index(scores.argmax(), scores.shape)
    return contact_enumerators[number][0](
        trace_, key,
        contact_param_grid[i], VARIANCE_GRID[j], OUTLIER_GRID[k]
    )
c2f_contact_update_jit = jax.jit(c2f_contact_update, static_argnames=("number",))

In [18]:
VARIANCE_GRID = jnp.array([0.001, 0.01, 0.01])
OUTLIER_GRID = jnp.array([0.0001, 0.001, 0.01])
# VARIANCE_GRID = jnp.array([0.001])
# OUTLIER_GRID = jnp.array([ 0.0001])

grid_params = [
    (0.2, jnp.pi, (11,11,11)), (0.1, jnp.pi/3, (11,11,11)), (0.05, 0.0, (11,11,1)),
    (0.05, jnp.pi/5, (11,11,11)), (0.02, 2*jnp.pi, (5,5,51)), (0.02, jnp.pi/5, (11,11,11))
]
contact_param_gridding_schedule = [
    b.utils.make_translation_grid_enumeration_3d(
        -x, -x, -ang,
        x, x, ang,
        *nums
    )
    for (x,ang,nums) in grid_params
]
key = jax.random.PRNGKey(500)


In [19]:
scene_id = 0

In [20]:
V_VARIANT = 0
O_VARIANT = 0
HIERARCHICAL_BAYES = True

if HIERARCHICAL_BAYES:
    V_GRID = VARIANCE_GRID
    O_GRID = OUTLIER_GRID
else:
    V_GRID, O_GRID = jnp.array([VARIANCE_GRID[V_VARIANT]]), jnp.array([OUTLIER_GRID[O_VARIANT]])

print(V_GRID, O_GRID)

gt_trace = importance_jit(key, *joblib.load(f"data/trace_{scene_id}.joblib"))[1][1]
choices = gt_trace.get_choices()
key, (_,trace) = importance_jit(key, choices, (jnp.arange(1), jnp.arange(22), *gt_trace.get_args()[2:]))
print(trace.get_score())

[0.001 0.01  0.01 ] [1.e-04 1.e-03 1.e-02]
-39228.242


In [21]:


all_all_paths = []
for _ in range(3):
    all_paths = []
    for obj_id in tqdm(range(len(b.RENDERER.meshes)-1)):
        path = []
        trace_ = add_object_jit(trace, key, obj_id, 0, 2,3)
        number = b.genjax.get_contact_params(trace_).shape[0] - 1
        path.append(trace_)
        for c2f_iter in range(len(contact_param_gridding_schedule)):
            trace_ = c2f_contact_update_jit(trace_, key, number,
                contact_param_gridding_schedule[c2f_iter], V_GRID, O_GRID)
            path.append(trace_)
        # for c2f_iter in range(len(contact_param_gridding_schedule)):
        #     trace_ = c2f_contact_update_jit(trace_, key, number,
        #         contact_param_gridding_schedule[c2f_iter], VARIANCE_GRID, OUTLIER_GRID)
        all_paths.append(
            path
        )
    all_all_paths.append(all_paths)
    
    scores = jnp.array([t[-1].get_score() for t in all_paths])
    print(scores)
    normalized_scores = b.utils.normalize_log_scores(scores)
    trace = all_paths[jnp.argmax(scores)][-1]

print(b.genjax.get_indices(gt_trace))
print(b.genjax.get_indices(trace))
b.genjax.viz_trace_meshcat(trace)

100%|████████████████████████████████████████████████████████████████████████████| 21/21 [00:04<00:00,  4.58it/s]


[-70671.94  -74412.69  -68695.2   -77906.555 -70077.47  -81027.47
 -79502.766 -81197.81  -76295.01  -79827.45  -73495.94  -68569.516
 -81002.266 -77031.15  -74384.51  -69481.26  -82187.625 -84797.09
 -80387.32  -78919.58  -80534.45 ]


100%|████████████████████████████████████████████████████████████████████████████| 21/21 [00:06<00:00,  3.34it/s]


[-38630.43  -44211.36  -37163.484 -31459.273 -34714.27  -28666.676
 -27164.193 -28841.414 -29099.914 -29611.127 -44377.742 -38100.92
 -31255.648 -28513.896 -28786.5   -40507.844 -30315.898 -32453.371
 -28863.953 -29039.38  -27914.383]


100%|████████████████████████████████████████████████████████████████████████████| 21/21 [00:08<00:00,  2.51it/s]

[-27790.014 -34558.46  -27424.463 -23734.63  -25744.498 -18513.54
 -19824.396 -18349.344 -22798.611 -18939.396 -33822.484 -26595.693
 -26220.236 -22523.234 -24566.227 -29832.549 -17487.96  -19971.922
 -16310.651 -17735.854 -18701.469]
[21 15 18  6]
[21 11  6 18]


In [13]:
b.genjax.viz_trace_meshcat(trace)

In [83]:
trace_alternate = all_paths[4][-1]
b.genjax.print_trace(trace)
b.genjax.print_trace(trace_alternate)


    SCORE: 152959.3750000
    VARIANCE: 0.0000100
    OUTLIER_PROB 0.0100000
    

    SCORE: 152818.5625000
    VARIANCE: 0.0000100
    OUTLIER_PROB 0.0100000
    
